# 📊 Аналіз Кореляції: Пульс ↔ Ліки

**Проект:** Medication Tracking App  
**Мета:** Виявити кореляцію між прийомом ліків та змінами пульсу  
**Дата:** Січень 2026  

---

## Зміст:
1. Завантаження даних
2. Exploratory Data Analysis (EDA)
3. Кореляційний аналіз
4. Візуалізація
5. Statistical Significance
6. Висновки

In [ ]:
# Імпорт бібліотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta

# Налаштування візуалізації
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Бібліотеки завантажено")

## 1️⃣ Завантаження Даних

In [ ]:
# Завантажуємо дані трекера (CSV експортований з додатку)
tracker_df = pd.read_csv('tracker_data.csv')
tracker_df['timestamp'] = pd.to_datetime(tracker_df['timestamp'])

# Завантажуємо дані ліків
meds_df = pd.read_csv('medication_log.csv')
meds_df['taken_at'] = pd.to_datetime(meds_df['taken_at'])

print(f"📊 Дані трекера: {len(tracker_df)} записів")
print(f"💊 Дані ліків: {len(meds_df)} записів")

# Перегляд даних
display(tracker_df.head())
display(meds_df.head())

## 2️⃣ Exploratory Data Analysis

In [ ]:
# Статистика пульсу
print("📈 Статистика пульсу:")
print(tracker_df['heart_rate'].describe())

# Розподіл пульсу
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(tracker_df['heart_rate'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Пульс (уд/хв)')
plt.ylabel('Частота')
plt.title('Розподіл Пульсу')

plt.subplot(1, 2, 2)
plt.boxplot(tracker_df['heart_rate'])
plt.ylabel('Пульс (уд/хв)')
plt.title('Box Plot Пульсу')

plt.tight_layout()
plt.show()

In [ ]:
# Пульс по годинах доби
tracker_df['hour'] = tracker_df['timestamp'].dt.hour

hourly_hr = tracker_df.groupby('hour')['heart_rate'].mean()

plt.figure(figsize=(12, 5))
plt.plot(hourly_hr.index, hourly_hr.values, marker='o', linewidth=2)
plt.xlabel('Година доби')
plt.ylabel('Середній пульс (уд/хв)')
plt.title('Циркадний Ритм Пульсу')
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24, 2))
plt.show()

print("💡 Спостерігаємо циркадний ритм - пульс нижчий вночі, вищий вдень")

## 3️⃣ Кореляційний Аналіз

In [ ]:
# Об'єднуємо дані: знаходимо чи був прийом ліків за останні N годин
def mark_medication_effect(tracker_row, meds_df, hours_window=6):
    """
    Перевіряє чи був прийом ліків за останні hours_window годин
    """
    timestamp = tracker_row['timestamp']
    window_start = timestamp - timedelta(hours=hours_window)
    
    # Знаходимо ліки прийняті в цьому вікні
    recent_meds = meds_df[
        (meds_df['taken_at'] >= window_start) & 
        (meds_df['taken_at'] <= timestamp)
    ]
    
    if len(recent_meds) > 0:
        # Повертаємо назву ліків та час після прийому
        first_med = recent_meds.iloc[0]
        hours_since = (timestamp - first_med['taken_at']).total_seconds() / 3600
        return first_med['medication_name'], hours_since
    return None, None

# Додаємо колонки
tracker_df[['recent_medication', 'hours_since_med']] = tracker_df.apply(
    lambda row: pd.Series(mark_medication_effect(row, meds_df)),
    axis=1
)

print(f"✅ Знайдено {tracker_df['recent_medication'].notna().sum()} записів з недавнім прийомом ліків")

In [ ]:
# Аналізуємо ефект кожного ліку окремо
medications = meds_df['medication_name'].unique()

results = []

for med_name in medications:
    # Пульс ПІСЛЯ прийому ліків (перші 6 годин)
    with_med = tracker_df[
        (tracker_df['recent_medication'] == med_name) & 
        (tracker_df['hours_since_med'] <= 6)
    ]['heart_rate']
    
    # Пульс БЕЗ прийому ліків
    without_med = tracker_df[
        tracker_df['recent_medication'].isna()
    ]['heart_rate']
    
    if len(with_med) > 5 and len(without_med) > 5:
        # Статистика
        mean_with = with_med.mean()
        mean_without = without_med.mean()
        change = mean_with - mean_without
        
        # t-test
        t_stat, p_value = stats.ttest_ind(with_med, without_med)
        
        # Кореляція Пірсона
        # Створюємо binary змінну (0 = без ліків, 1 = з ліками)
        temp_df = tracker_df.copy()
        temp_df['med_binary'] = (temp_df['recent_medication'] == med_name).astype(int)
        correlation, corr_p = stats.pearsonr(
            temp_df['med_binary'], 
            temp_df['heart_rate']
        )
        
        results.append({
            'Ліки': med_name,
            'Середній пульс З ліками': round(mean_with, 1),
            'Середній пульс БЕЗ ліків': round(mean_without, 1),
            'Зміна пульсу': round(change, 1),
            'Кореляція': round(correlation, 3),
            'P-value': round(p_value, 4),
            'Значущий?': '✅ Так' if p_value < 0.05 else '❌ Ні',
            'Кількість вимірів': len(with_med)
        })

results_df = pd.DataFrame(results)
display(results_df)

## 4️⃣ Візуалізація Ефектів

In [ ]:
# Графік ефектів кожного ліку
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, med_name in enumerate(medications[:4]):  # Перші 4 ліки
    ax = axes[idx]
    
    # Дані з ліками
    med_data = tracker_df[
        (tracker_df['recent_medication'] == med_name) & 
        (tracker_df['hours_since_med'] <= 6)
    ]
    
    if len(med_data) > 0:
        # Групуємо по годинах після прийому
        hourly_effect = med_data.groupby(
            pd.cut(med_data['hours_since_med'], bins=[0, 1, 2, 3, 4, 5, 6])
        )['heart_rate'].mean()
        
        # Базовий пульс (без ліків)
        baseline = tracker_df[tracker_df['recent_medication'].isna()]['heart_rate'].mean()
        
        ax.axhline(y=baseline, color='gray', linestyle='--', label='Базовий пульс', linewidth=2)
        ax.plot(range(len(hourly_effect)), hourly_effect.values, 
                marker='o', linewidth=3, markersize=8, label=med_name)
        
        ax.set_xlabel('Години після прийому')
        ax.set_ylabel('Пульс (уд/хв)')
        ax.set_title(f'Ефект {med_name} на пульс')
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots порівняння
fig, ax = plt.subplots(figsize=(12, 6))

data_for_boxplot = []
labels = []

for med_name in medications:
    with_med = tracker_df[
        (tracker_df['recent_medication'] == med_name) & 
        (tracker_df['hours_since_med'] <= 6)
    ]['heart_rate']
    
    if len(with_med) > 5:
        data_for_boxplot.append(with_med)
        labels.append(med_name)

# Додаємо "без ліків"
without_med = tracker_df[tracker_df['recent_medication'].isna()]['heart_rate']
data_for_boxplot.append(without_med)
labels.append('БЕЗ ЛІКІВ')

ax.boxplot(data_for_boxplot, labels=labels)
ax.set_ylabel('Пульс (уд/хв)')
ax.set_title('Розподіл Пульсу: З Ліками vs Без Ліків')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 5️⃣ Statistical Significance

In [ ]:
# Тепла карта кореляцій
print("📊 Кореляційна Матриця:")
print("\nІнтерпретація:")
print("• Кореляція > 0.5: Сильна позитивна (пульс ЗРОСТАЄ)")
print("• Кореляція < -0.5: Сильна негативна (пульс ПАДАЄ)")
print("• |Кореляція| < 0.3: Слабкий зв'язок")
print("• P-value < 0.05: Статистично значущий результат ✅")
print("\n")

# Виводимо результати з інтерпретацією
for _, row in results_df.iterrows():
    print(f"\n{'='*60}")
    print(f"💊 {row['Ліки']}")
    print(f"{'='*60}")
    print(f"Середній пульс З ліками:   {row['Середній пульс З ліками']} уд/хв")
    print(f"Середній пульс БЕЗ ліків:  {row['Середній пульс БЕЗ ліків']} уд/хв")
    print(f"Зміна пульсу:              {row['Зміна пульсу']:+.1f} уд/хв")
    print(f"Кореляція:                 {row['Кореляція']:.3f}")
    print(f"P-value:                   {row['P-value']:.4f}")
    print(f"Статистично значущий:      {row['Значущий?']}")
    print(f"Кількість вимірів:         {row['Кількість вимірів']}")
    
    # Інтерпретація
    if row['P-value'] < 0.05:
        if row['Зміна пульсу'] < -5:
            print(f"\n💡 Висновок: {row['Ліки']} ЗНИЖУЄ пульс на {abs(row['Зміна пульсу']):.1f} уд/хв")
        elif row['Зміна пульсу'] > 5:
            print(f"\n💡 Висновок: {row['Ліки']} ПІДВИЩУЄ пульс на {row['Зміна пульсу']:.1f} уд/хв")
        else:
            print(f"\n💡 Висновок: {row['Ліки']} має НЕВЕЛИКИЙ вплив на пульс")
    else:
        print(f"\n💡 Висновок: Недостатньо даних для висновків про {row['Ліки']}")

## 6️⃣ Фінальні Висновки

In [ ]:
print("="*80)
print("📋 ФІНАЛЬНИЙ ЗВІТ")
print("="*80)

# Загальна статистика
print(f"\n📊 Загальна Статистика:")
print(f"   • Аналізовано днів: {(tracker_df['timestamp'].max() - tracker_df['timestamp'].min()).days}")
print(f"   • Всього вимірів пульсу: {len(tracker_df)}")
print(f"   • Всього прийомів ліків: {len(meds_df)}")
print(f"   • Середній пульс: {tracker_df['heart_rate'].mean():.1f} ± {tracker_df['heart_rate'].std():.1f} уд/хв")

# Топ-3 ліки з найбільшим впливом
print(f"\n🏆 Топ-3 Ліки з Найбільшим Впливом на Пульс:")
top_meds = results_df.sort_values('Зміна пульсу', key=abs, ascending=False).head(3)
for idx, (_, row) in enumerate(top_meds.iterrows(), 1):
    print(f"   {idx}. {row['Ліки']}: {row['Зміна пульсу']:+.1f} уд/хв (p={row['P-value']:.4f})")

# Рекомендації
print(f"\n💡 Рекомендації:")
for _, row in results_df.iterrows():
    if row['P-value'] < 0.05 and abs(row['Зміна пульсу']) > 5:
        if row['Зміна пульсу'] < 0:
            print(f"   • {row['Ліки']}: Контролювати пульс після прийому (знижується)")
        else:
            print(f"   • {row['Ліки']}: ⚠️ УВАГА! Підвищує пульс - консультація з лікарем")

print("\n" + "="*80)
print("✅ Аналіз завершено!")
print("="*80)

---

## 📚 Додатково: Експорт Результатів

In [ ]:
# Зберегти результати в CSV
results_df.to_csv('correlation_results.csv', index=False)
print("✅ Результати збережено в correlation_results.csv")

# Зберегти графіки
# plt.savefig('heart_rate_analysis.png', dpi=300, bbox_inches='tight')
# print("✅ Графіки збережено в heart_rate_analysis.png")